In [ ]:
!pip install sentence-transformers==2.7.0 datasets==2.19.0 huggingface_hub==0.23.0 faiss-cpu==1.8.0 -qqq

# **사전 학습된 언어 모델을 불러와 문장 임베딩 모델 만들기**

In [ ]:
from sentence_transformers import SentenceTransformer, models

# 허깅페이스 임베딩 모델 로드
transformer_model = models.Transformer('klue/roberta-base')

# 풀링층 생성
pooling_layer = models.Pooling(
    transformer_model.get_word_embedding_dimension(),
    # 평균 풀링 모드
    pooling_mode_mean_tokens=True
)

embedding_model = SentenceTransformer(modules=[transformer_model, pooling_layer])

# **실습 데이터셋 다운로드 및 확인**

In [ ]:
from datasets import load_dataset

klue_sts_train = load_dataset('klue', 'sts', split='train')
klue_sts_test = load_dataset('klue', 'sts', split='validation')
klue_sts_train[0]

# {'guid': 'klue-sts-v1_train_00000',
#  'source': 'airbnb-rtt',
#  'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
#  'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
#  'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

# **학습 데이터에서 검증 데이터셋 분리하기**

In [ ]:
# 학습 데이터셋의 10%를 검증 데이터셋으로 구성한다.
klue_sts_train = klue_sts_train.train_test_split(test_size=0.1, seed=42)
klue_sts_train, klue_sts_eval = klue_sts_train['train'], klue_sts_train['test']

# **label 정규화하기**

In [ ]:
from sentence_transformers import InputExample

# 유사도 점수를 0~1 사이로 정규화 하고 InputExample 객체에 담는다.
# STS 학습 방법을 위해 InputExample 객체 사용 필요
def prepare_sts_examples(dataset):
  examples = []

  for data in dataset:
    examples.append(
        InputExample(
            texts=[data['sentence1'], data['sentence2']],
            # 데이터셋의 유사도 점수는 0 ~ 5 범위를 가짐
            # Sentence-BERT 모델에서는 유사도 점수를 0 ~ 1 사이의 값으로 사용해야 함
            # 따라서 5로 나눠
            label=data['labels']['label'] / 5.0
        )
    )

  return examples

train_examples = prepare_sts_examples(klue_sts_train)
eval_examples = prepare_sts_examples(klue_sts_eval)
test_examples = prepare_sts_examples(klue_sts_test)